In [2]:
pip install kagglehub

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Nithish\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import kagglehub

# Download latest version of the dataset
path = kagglehub.dataset_download("tapakah68/hand-gesture-recognition-dataset")

print("Path to dataset files:", path)


C:\Users\Nithish\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2


In [9]:
import os

dataset_path = r"C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2"
for root, dirs, files in os.walk(dataset_path):
    print(f"Root: {root}")
    print(f"Directories: {dirs}")
    print(f"Files: {files}")


Root: C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2
Directories: ['files']
Files: ['hand_gestures.csv']
Root: C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2\files
Directories: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '3', '4', '5', '6', '7', '8', '9']
Files: []
Root: C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2\files\0
Directories: []
Files: ['fist.mp4', 'four.mp4', 'me.mp4', 'one.mp4', 'small.mp4']
Root: C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2\files\1
Directories: []
Files: ['fist.mp4', 'four.mp4', 'me.mp4', 'one.mp4', 'small.mp4']
Root: C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2\files\10
Directories: []
Files: ['fist.mp4', 'four.mp4', 'me.mp4'

In [10]:
dataset_path = r"C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2"


In [13]:
if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
    pass  # Placeholder for future code


In [15]:
gesture_classes = os.listdir(dataset_path)  # Gesture class directories or files
for gesture_class in gesture_classes:
    gesture_folder = os.path.join(dataset_path, gesture_class)
    if os.path.isdir(gesture_folder):  # Proceed only if it's a directory
        for image_file in os.listdir(gesture_folder):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(gesture_folder, image_file)
                landmarks = extract_landmarks_from_image(image_path)
                if landmarks:
                    X.append(landmarks)
                    y.append(gesture_class)
    else:
        print(f"Skipping non-directory: {gesture_folder}")  # Optional debug


Skipping non-directory: C:\Users\Nithish\.cache\kagglehub\datasets\tapakah68\hand-gesture-recognition-dataset\versions\2\hand_gestures.csv


In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import joblib
import numpy as np

# Load your trained Random Forest model
model_path = 'path_to_your_trained_model.pkl'
clf = joblib.load(model_path)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Get the screen size
screen_width, screen_height = pyautogui.size()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a selfie-view display
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw hand landmarks on the frame
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the landmark coordinates for the index finger tip (landmark 8)
            index_finger_tip = hand_landmarks.landmark[8]
            x, y = int(index_finger_tip.x * screen_width), int(index_finger_tip.y * screen_height)

            # Extract the region of interest (ROI) around the finger tip
            roi_size = 64
            half_size = roi_size // 2
            roi = frame[max(0, y - half_size):min(frame.shape[0], y + half_size),
                        max(0, x - half_size):min(frame.shape[1], x + half_size)]
            
            if roi.shape[0] == roi_size and roi.shape[1] == roi_size:
                # Preprocess ROI
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                roi_flat = roi_gray.flatten().reshape(1, -1)

                # Predict with the trained model
                prediction = clf.predict(roi_flat)

                # Use the prediction to control the cursor (example: use the centroid of the hand)
                pyautogui.moveTo(x, y)
                print(f"Prediction: {prediction}")

    # Display the frame
    cv2.imshow('Hand Tracking', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


Folder path: C:\Users\Nithish\OneDrive\Desktop\Prog\mini\Code\files\0
Processing folder: 0
Video path: C:\Users\Nithish\OneDrive\Desktop\Prog\mini\Code\files\0\fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame from video: fist.mp4
Extracted frame f